In [20]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(response.text, 'lxml')

table = soup.find('table', {'class':'wikitable sortable'})
table_rows = table.find_all('tr')

canada_list = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    canada_list.append(row)
    
df = pd.DataFrame(canada_list, columns=['PostalCode', 'Borough', 'Neighbourhood'])

df['Neighbourhood'] = df['Neighbourhood'].str.strip()

df.Borough.replace('Not assigned', np.nan, inplace=True)

df.Neighbourhood.replace('Not assigned', df['Borough'], inplace=True)

df.dropna(subset=['Borough'], inplace=True)

canada_df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()

canada_df.shape


(103, 3)

In [ ]:
#!conda install -c conda-forge geocoder -y
# import geocoder 

# loop until you get the coordinates
# g = geocoder.google('{}, Toronto, Ontario'.format('M1B'))

# print (g)

In [14]:
geocode_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [22]:
clean_df = pd.merge(canada_df, geocode_df, left_on='PostalCode', right_on='Postal Code', how='left')

clean_df.drop(['Postal Code'], axis=1, inplace=True)

clean_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
